In [10]:
import pickle
import torch
import pandas as pd
from tqdm import tqdm
import numpy as np
from scipy.spatial.distance import cdist

import sys
sys.path.append("..")
import clustering

from transformers import BertTokenizer

In [2]:
feature_set = 'binder'

In [5]:
with open('../data/cwr4lsc/{}/usages_len128.clustering.2.dict'.format(feature_set), 'rb') as f:
    clusters = pickle.load(f)

these are the clusters for virtual in coha

In [6]:
clusters['virtual']

KMeans(n_clusters=2, random_state=42)

mario has provided us with a function to sample the most prototypical usages from the cluster

for that we need to load the usages

In [7]:
with open('../data/cwr4lsc/{}/matrix_usages_16_len128.dict'.format(feature_set), 'rb') as f:
    usages = pickle.load(f)

Now I want to get the clusterid for each usage, and normalize the data into tidy format

let's do a test to get the clusterid for a usage

In [8]:
U_w, contexts, positions, t_labels = usages['virtual']
#example_usage = U_w[1231]

In [9]:
preds = clusters['virtual'].predict(U_w)
clusters['virtual'].cluster_centers_

clusters['virtual'].labels_.shape

(1940,)

Let's look at the data a little bit with 

In [11]:
# word = 'virtual'
# window = 5
# U_w, contexts, positions, t_labels = usages[word]
# cluster_id = 1
# n = 10


# print(clusters['virtual'].cluster_centers_.shape)

# centers = clusters['virtual'].cluster_centers_

# print("shape of labels: ", clusters['virtual'].labels_.shape)

# labels = clusters['virtual'].labels_

# #print("shape of usages: ", usages['virtual'].shape)
# print("number of usages: ", usages['virtual'][0].shape)


# nearest = np.argsort(cdist([clusters['virtual'].cluster_centers_[cluster_id]], usages['virtual'][0]), axis=1)[:, -n:]

# print(nearest[0])


In [12]:
U_w, contexts, positions, t_labels = usages['virtual']
print(U_w[:5])
print(contexts[:5])
print(positions[:5])
print(t_labels[:5])

[[ 1.37155902  0.74701113  0.68513262  0.88769364  0.31982324 -0.5443539
   0.98743403 -0.41255775 -0.79178655 -0.34121916 -0.06430614  0.01541085
  -0.72087657 -0.98633212  0.35177878 -0.35466141  0.04431364  0.15492362
  -0.0988112  -0.93392903 -1.0632689  -0.34798449 -0.22970758 -0.37942541
  -0.01137507 -0.76013094 -0.8618564  -0.37450206 -1.25279641 -0.37910265
  -0.34031749  0.55303037 -0.04993583  1.11559749  0.3623181  -0.5534032
   0.08852845  0.30130619  0.7064231   0.74368632  1.00611794  0.04866311
   0.57447565  0.86512351 -0.58402896  0.94512302  0.80908138  1.97223032
   0.18008894 -0.05232576  0.73623115  0.62196225  0.55818421  0.69126391
   0.38810408  0.41157055  0.15244736  0.21383613  0.79748726  0.35449287
   0.55290926  0.49548233]
 [ 1.73348939  1.10719681  0.4653365   1.30692697  0.67544162 -0.27515912
   0.7078892   0.04294072 -0.44345418 -0.14074402 -0.0169379   1.01380873
  -0.35449001 -0.97559756  0.54417896 -0.19709367  0.58421969  0.53839976
  -0.60668087

In [13]:
# Here's the number of usages in each cluster

from collections import Counter
Counter(clusters['virtual'].labels_)

Counter({0: 1097, 1: 843})

In [ ]:
now we remap the data to have one row for each feature

In [14]:
# get the feature labels for the model
model = torch.load('../trained_models/{}'.format(feature_set))
feature_map = model.feature_norms.feature_map
feature_labels = [str(feature_map.get_object(i)) for i in range(0, len(feature_map))]
print(feature_labels[:10])



['Vision', 'Bright', 'Dark', 'Color', 'Pattern', 'Large', 'Small', 'Motion', 'Biomotion', 'Fast']


In [15]:
# model dimensions dont have full 65 oops who knows what happened if our original downloaded data was bunk
len(feature_labels)



62

In [17]:
words = []
years = []
prototypes =[]
usage_ids = []
#positions = []
features = []
values = []

# iterate through the words
for word in clusters.keys():
    # get the usages for this word, where the usage is a tuple of matrices with feature vector and other info
    U_w, contexts, positions, t_labels = usages[word]
    predictions = clusters[word].predict(U_w)
    
    for usage_id, u_w in tqdm(enumerate(U_w)):

        i=0
        for value in u_w:
            words.append(word)
            years.append(t_labels[usage_id])
            prototypes.append(predictions[usage_id])
            usage_ids.append(usage_id)
            features.append(feature_labels[i])
            values.append(value)
            i+=1

        #prototypes = clustering.get_prototypes(word, clusters[word], usages[word], n=5, window=10)

    tidy_df = pd.DataFrame.from_records(
        {"word": words,
         "year": years,
         "prototype": prototypes, 
         "usage_id": usage_ids,
        #"prototype_sentences": prototypes, 
        "feature": features,
        "value": values,
        }
    )
    tidy_df.to_csv('../data/cwr4lsc/{}/{}_features.csv'.format(feature_set, word))

    

6668it [00:00, 35905.27it/s]
32448it [00:00, 34864.23it/s]
60311it [00:01, 35679.87it/s]
22451it [00:00, 34873.97it/s]
10169it [00:00, 35073.65it/s]
6285it [00:00, 36380.65it/s]
7834it [00:00, 33669.58it/s]
7521it [00:00, 35954.74it/s]
2804it [00:00, 36964.79it/s]
37883it [00:01, 36275.18it/s]
13155it [00:00, 34774.87it/s]
1954it [00:00, 37315.46it/s]
11622it [00:00, 35833.55it/s]
14381it [00:00, 32497.52it/s]
3068it [00:00, 36551.98it/s]
1940it [00:00, 37119.94it/s]
2621it [00:00, 36675.04it/s]
2486it [00:00, 37406.96it/s]
1183it [00:00, 36497.43it/s]


In [36]:
tidy_df.head(5)

,feature,prototype_id,target,value
0,desert,0,leaf,-1.186869
1,give,0,leaf,-1.121639
2,leave,0,leaf,-1.951784
3,up,0,leaf,0.149296
4,withdraw,0,leaf,-1.602879
